In [1]:
# 2+ stores TA zips to removed selected in Tableau

import pandas as pd
import numpy as np
import os
import datetime
from haversine import haversine
import json

zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [2]:
remove_zip_2_store_TA_dict={"1":"98012",
                           "14":"89036",
                           "87":"20817",
                           "7":"55446"}

In [3]:
US_ta_excel=pd.ExcelFile("/home/jian/Projects/Savers/TA_creation/TA_output/Savers_US_TA_2nd_Version_JL_2019-03-14.xlsx")
US_ta_excel.sheet_names

['Store_list',
 'revised_TA_Summary',
 'TA_Summary_by_store_count',
 'TA_zips',
 'tableau_data',
 'maps_2_plus_stores_TA',
 'maps_1_store_TA_5_miles_plus']

In [4]:
tab_1_Store_list=US_ta_excel.parse("Store_list",dtype=str)

tab_2_revised_TA_Summary=US_ta_excel.parse("revised_TA_Summary",dtype=str)
tab_2_revised_TA_Summary['store_count']=tab_2_revised_TA_Summary['store_count'].astype(int)

tab_3_TA_Summary_by_store_count=US_ta_excel.parse("TA_Summary_by_store_count",dtype=str)
tab_4_TA_zips=US_ta_excel.parse("TA_zips",dtype=str)
tab_5_tableau_data=US_ta_excel.parse("tableau_data",dtype=str)
tab_6_maps_2_plus_stores_TA=US_ta_excel.parse("maps_2_plus_stores_TA",dtype=str)
tab_7_maps_1_store_TA_5_miles_plus=US_ta_excel.parse("maps_1_store_TA_5_miles_plus",dtype=str)


In [5]:
store_center=tab_1_Store_list[['location_id','Google_lat','Google_lng']].drop_duplicates()
store_center['Google_lat']=store_center['Google_lat'].astype(float)
store_center['Google_lng']=store_center['Google_lng'].astype(float)
store_center['store_center']=store_center[['Google_lat','Google_lng']].values.tolist()
store_center_dict=store_center.set_index("location_id").to_dict()['store_center']

In [6]:
tab_7_maps_1_store_TA_5_miles_plus['dist_max']=tab_7_maps_1_store_TA_5_miles_plus['dist_max'].astype(float)
tab_7_maps_1_store_TA_5_miles_plus_keep=tab_7_maps_1_store_TA_5_miles_plus[tab_7_maps_1_store_TA_5_miles_plus['dist_max']<5.6]

tab_7_maps_1_store_TA_5_miles_plus_revise=tab_7_maps_1_store_TA_5_miles_plus[tab_7_maps_1_store_TA_5_miles_plus['dist_max']>5.6]



In [7]:
revise_TA_list=tab_7_maps_1_store_TA_5_miles_plus_revise['ta_number'].tolist()

tab_2_revised_TA_Summary_revise=tab_2_revised_TA_Summary[tab_2_revised_TA_Summary['ta_number'].isin(revise_TA_list)]
tab_2_revised_TA_Summary_revise['zip_list']=tab_2_revised_TA_Summary_revise['zip_list'].apply(lambda x: eval(x))
tab_2_revised_TA_Summary_revise['store_list']=tab_2_revised_TA_Summary_revise['store_list'].apply(lambda x: eval(x))


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
tab_2_revised_TA_Summary_revise['dist_list']=np.nan
tab_2_revised_TA_Summary_revise['dist_max']=np.nan
tab_2_revised_TA_Summary_revise['dist_median']=np.nan
tab_2_revised_TA_Summary_revise['dist_mean']=np.nan
tab_2_revised_TA_Summary_revise['dist_min']=np.nan
tab_2_revised_TA_Summary_revise['maxium_zip']=np.nan

for index,row in tab_2_revised_TA_Summary_revise.iterrows():
    zip_list=row['zip_list']
    dist_list=[]
    store=list(row['store_list'])[0]
    store_center_coor=store_center_dict[store]
    
    dist_max=0
    zip_pair_max_dist=[]
    
    for i in range(len(zip_list)):
        for j in range(i+1,len(zip_list)):
            zip_i=zip_list[i]
            zip_j=zip_list[j]
            
            dist=haversine(zip_centers[zip_i],zip_centers[zip_j],miles=True)
            dist_list.append(dist)
            if dist>dist_max:
                dist_max=dist
                zip_pair_max_dist=[zip_i,zip_j]
                

    dist_i=haversine(zip_centers[zip_pair_max_dist[0]],store_center_coor,miles=True)
    dist_j=haversine(zip_centers[zip_pair_max_dist[1]],store_center_coor,miles=True)
    if dist_i>=dist_j:
        zip_max=zip_pair_max_dist[0]
    else:
        zip_max=zip_pair_max_dist[1]
    zip_list.remove(zip_max)
    if len(dist_list)>=1:
        # tab_2_revised_TA_Summary_revise.loc[index, 'dist_list']=str(dist_list)
        # tab_2_revised_TA_Summary_revise.loc[index,'dist_max']=np.max(dist_list)
        # tab_2_revised_TA_Summary_revise.loc[index,'dist_median']=np.median(dist_list)
        # tab_2_revised_TA_Summary_revise.loc[index,'dist_mean']=np.mean(dist_list)
        # tab_2_revised_TA_Summary_revise.loc[index,'dist_min']=np.min(dist_list)
        tab_2_revised_TA_Summary_revise.loc[index,'maxium_zip']=zip_max


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [9]:
remove_zip_1_store_TA_dict=tab_2_revised_TA_Summary_revise.set_index("ta_number").to_dict()['maxium_zip']
len(remove_zip_1_store_TA_dict)

98

In [10]:
all_ta_remove_zip_dict=remove_zip_1_store_TA_dict.copy()
all_ta_remove_zip_dict.update(remove_zip_2_store_TA_dict)

In [11]:
len(all_ta_remove_zip_dict)

del tab_2_revised_TA_Summary_revise

In [12]:
tab_1_Store_list=tab_1_Store_list.rename(columns={"zip_list_in_5_miles":"zip_list_in_5_miles_original"})
tab_1_Store_list['zip_list_in_5_miles_original']=tab_1_Store_list['zip_list_in_5_miles_original'].apply(lambda x: eval(x))
tab_1_Store_list['zip_list_in_5_miles_new']=np.nan

In [13]:

for i in range(len(tab_1_Store_list)):
    zip_list_str=str(tab_1_Store_list.loc[i,'zip_list_in_5_miles_original'])
    
    if tab_1_Store_list.loc[i,'ta_number'] in all_ta_remove_zip_dict.keys():
        
        zip_list_copy=eval(zip_list_str)
        TA_num=tab_1_Store_list.loc[i,'ta_number']
        if all_ta_remove_zip_dict[TA_num] in zip_list_copy:
            # print(i)
            zip_list_copy.remove(all_ta_remove_zip_dict[TA_num])
        tab_1_Store_list.loc[i,'zip_list_in_5_miles_new']=str(zip_list_copy)
    else:
        tab_1_Store_list.loc[i,'zip_list_in_5_miles_new']=zip_list_str

tab_1_Store_list.head(2)

,location_id,Name/Indicator,Brand,Address,City,St/Province,Zip/Postal,Country,Company,Phone,...,Cluster,Search_Key_Brand,Google_Address,Google_lat,Google_lng,store_zip,zip_list_in_5_miles_original,ta_number,TA_label,zip_list_in_5_miles_new
0,1003,Burien,Value Village,131 SW 157th St,Burien,WA,98166,US,TVI,206-246-6237,...,3,"Value Village, 131 SW 157th St, Burien, WA, 98166","131 SW 157th St, Burien, WA 98166, USA",47.4618219,-122.3354587,98166,"[98166, 98062, 98158, 98148, 98188, 98198, 981...",91,"2_{'1173', '1003'}","['98166', '98062', '98158', '98148', '98188', ..."
1,1004,Lake City,Value Village,12548 Lake City Way NE,Seattle,WA,98125,US,TVI,(206) 365-8232,...,3,"Value Village, 12548 Lake City Way NE, Seattle...","12548 Lake City Way NE, Seattle, WA 98125, USA",47.7205966,-122.2937321,98125,"[98028, 98195, 98145, 98133, 98115, 98105, 981...",1,"9_{'1111', '1125', '1004', '1214', '1021', '10...","['98028', '98195', '98145', '98133', '98115', ..."


In [14]:
tab_1_Store_list['len_zip_original']=tab_1_Store_list['zip_list_in_5_miles_original'].apply(lambda x: len(x))
tab_1_Store_list['len_zip_new']=tab_1_Store_list['zip_list_in_5_miles_new'].apply(lambda x: len(eval(x)))

# TAB 2

In [15]:
tab_2_revised_TA_Summary.head(2)

,ta_number,store_count,zip_count,store_list,zip_list,Cluster,dist_list,dist_max,dist_median,dist_mean,dist_min,ratio_max_to_median,city_list,state_list,DMA_list,Primary_City,Primary_State,Primary_DMA,TA_label
0,1,9,56,"{'1111', '1125', '1004', '1214', '1021', '1023...","['98028', '98195', '98145', '98133', '98115', ...",3,"[7.829780079742904, 7.4605886535748125, 4.7824...",33.59139167678625,10.6911154947475,11.52132643272832,0,3.141991281760034,"{'SEATTLE (WA)', 'REDMOND (WA)', 'EVERETT (WA)...",{'WA'},{'SEATTLE-TACOMA'},SEATTLE (WA),WA,SEATTLE-TACOMA,"9_{'1111', '1125', '1004', '1214', '1021', '10..."
1,2,1,13,{'1007'},"['94065', '95155', '94301', '94302', '94062', ...",4,"[7.7606192566939125, 8.196608020113727, 7.7522...",9.260507273812983,3.545390934778147,3.963251384386047,0.3765233218180234,2.611984811878711,"{'PALO ALTO (CA)', 'MENLO PARK (CA)', 'BELMONT...",{'CA'},{'SAN FRANCISCO-OAK-SAN JOSE'},REDWOOD CITY (CA),CA,SAN FRANCISCO-OAK-SAN JOSE,1_{'1007'}


In [16]:
tab_2_revised_TA_Summary=tab_2_revised_TA_Summary.rename(columns={"zip_list":"zip_list_original"})
tab_2_revised_TA_Summary['zip_list_new']=np.nan

for i in range(len(tab_2_revised_TA_Summary)):
    zip_list_str=str(tab_2_revised_TA_Summary.loc[i,'zip_list_original'])
    
    if tab_2_revised_TA_Summary.loc[i,'ta_number'] in all_ta_remove_zip_dict.keys():
        
        zip_list_copy=eval(zip_list_str)
        TA_num=tab_2_revised_TA_Summary.loc[i,'ta_number']
        if all_ta_remove_zip_dict[TA_num] in zip_list_copy:
            # print(i)
            zip_list_copy.remove(all_ta_remove_zip_dict[TA_num])
        tab_2_revised_TA_Summary.loc[i,'zip_list_new']=str(zip_list_copy)
    else:
        tab_2_revised_TA_Summary.loc[i,'zip_list_new']=zip_list_str

tab_2_revised_TA_Summary.head(2)

,ta_number,store_count,zip_count,store_list,zip_list_original,Cluster,dist_list,dist_max,dist_median,dist_mean,dist_min,ratio_max_to_median,city_list,state_list,DMA_list,Primary_City,Primary_State,Primary_DMA,TA_label,zip_list_new
0,1,9,56,"{'1111', '1125', '1004', '1214', '1021', '1023...","['98028', '98195', '98145', '98133', '98115', ...",3,"[7.829780079742904, 7.4605886535748125, 4.7824...",33.59139167678625,10.6911154947475,11.52132643272832,0,3.141991281760034,"{'SEATTLE (WA)', 'REDMOND (WA)', 'EVERETT (WA)...",{'WA'},{'SEATTLE-TACOMA'},SEATTLE (WA),WA,SEATTLE-TACOMA,"9_{'1111', '1125', '1004', '1214', '1021', '10...","['98028', '98195', '98145', '98133', '98115', ..."
1,2,1,13,{'1007'},"['94065', '95155', '94301', '94302', '94062', ...",4,"[7.7606192566939125, 8.196608020113727, 7.7522...",9.260507273812983,3.545390934778147,3.963251384386047,0.3765233218180234,2.611984811878711,"{'PALO ALTO (CA)', 'MENLO PARK (CA)', 'BELMONT...",{'CA'},{'SAN FRANCISCO-OAK-SAN JOSE'},REDWOOD CITY (CA),CA,SAN FRANCISCO-OAK-SAN JOSE,1_{'1007'},"['94065', '95155', '94302', '94062', '94061', ..."


In [17]:
tab_2_revised_TA_Summary=tab_2_revised_TA_Summary.rename(columns={"zip_count":"zip_count_original"})

tab_2_revised_TA_Summary['zip_count_new']=tab_2_revised_TA_Summary['zip_list_new'].apply(lambda x: len(eval(x)))


In [18]:
tab_2_revised_TA_Summary['dist_list']=np.nan
tab_2_revised_TA_Summary['dist_max']=np.nan
tab_2_revised_TA_Summary['dist_median']=np.nan
tab_2_revised_TA_Summary['dist_mean']=np.nan
tab_2_revised_TA_Summary['dist_min']=np.nan

for index,row in tab_2_revised_TA_Summary.iterrows():
    zip_list=eval(row['zip_list_new'])
    dist_list=[]
    for i in range(len(zip_list)):
        for j in range(i+1,len(zip_list)):
            zip_i=zip_list[i]
            zip_j=zip_list[j]
            
            dist=haversine(zip_centers[zip_i],zip_centers[zip_j],miles=True)
            dist_list.append(dist)
    if len(dist_list)>=1:
        tab_2_revised_TA_Summary.loc[index, 'dist_list']=str(dist_list)
        tab_2_revised_TA_Summary.loc[index,'dist_max']=np.max(dist_list)
        tab_2_revised_TA_Summary.loc[index,'dist_median']=np.median(dist_list)
        tab_2_revised_TA_Summary.loc[index,'dist_mean']=np.mean(dist_list)
        tab_2_revised_TA_Summary.loc[index,'dist_min']=np.min(dist_list)
        
tab_2_revised_TA_Summary['ratio_max_to_median']=tab_2_revised_TA_Summary['dist_max']/tab_2_revised_TA_Summary['dist_median']


# TAB 3

In [19]:
tab_3_TA_Summary_by_store_count.head(2)

# tab_3_TA_Summary_by_store_count       no change

,store_count,TA_count,TA_list
0,1,109,"['2', '3', '4', '5', '6', '8', '9', '10', '11'..."
1,2,8,"['39', '45', '58', '64', '91', '100', '101', '..."


# TAB 4

In [20]:
tab_4_TA_zips.shape

(1952, 5)

In [21]:
tab_4_TA_zips_updated=pd.DataFrame()
for ta_number,group in tab_4_TA_zips.groupby(['ta_number']):
    if ta_number in all_ta_remove_zip_dict.keys():
        group=group[group['zip_cd']!=all_ta_remove_zip_dict[ta_number]]
    tab_4_TA_zips_updated=tab_4_TA_zips_updated.append(group)

In [22]:
tab_4_TA_zips=tab_4_TA_zips_updated.copy()

# TAB 5

In [23]:
tab_5_tableau_data.head(2)

,Google_lat,Google_lng,TA_label,location_id,ta_number,zip_cd,dist_max,store_count
0,47.4618219,-122.3354587,"2_{'1173', '1003'}",1003,91,nan,10.67035746840996,2
1,47.7205966,-122.2937321,"9_{'1111', '1125', '1004', '1214', '1021', '10...",1004,1,nan,33.59139167678625,9


In [24]:
tab_5_tableau_data_1=tab_5_tableau_data[tab_5_tableau_data['Google_lat']!="0"]
tab_5_tableau_data_1['zip_cd']=tab_5_tableau_data_1['zip_cd'].replace("nan",np.nan)
tab_5_tableau_data_2=tab_4_TA_zips[['ta_number','TA_label','zip_cd']].drop_duplicates()

tab_5_tableau_data=tab_5_tableau_data_1.append(tab_5_tableau_data_2)
tab_5_tableau_data['Google_lat']=tab_5_tableau_data['Google_lat'].fillna("0")
tab_5_tableau_data['Google_lng']=tab_5_tableau_data['Google_lng'].fillna("0")
tab_5_tableau_data['Google_lat']=tab_5_tableau_data['Google_lat'].astype(float)
tab_5_tableau_data['Google_lat']=tab_5_tableau_data['Google_lat'].astype(float)
del tab_5_tableau_data['dist_max']
del tab_5_tableau_data['store_count']

df_TA_dist_max_count=tab_2_revised_TA_Summary[['ta_number','dist_max','store_count']]
tab_5_tableau_data=pd.merge(tab_5_tableau_data,df_TA_dist_max_count,on="ta_number",how="left")

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [25]:
tab_6_maps_2_plus_stores_TA=tab_2_revised_TA_Summary[tab_2_revised_TA_Summary['store_count']>=2]
tab_7_maps_1_store_TA_5_miles_plus=tab_2_revised_TA_Summary[(tab_2_revised_TA_Summary['store_count']==1) & (tab_2_revised_TA_Summary['dist_max']>5)]


In [26]:
writer=pd.ExcelWriter("/home/jian/Projects/Savers/TA_creation/TA_output/Savers_US_TA_3rd_Version_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

tab_1_Store_list.to_excel(writer,"Store_list",index=False)
tab_2_revised_TA_Summary.to_excel(writer,"revised_TA_Summary",index=False)
tab_3_TA_Summary_by_store_count.to_excel(writer,"TA_Summary_by_store_count",index=False)
tab_4_TA_zips.to_excel(writer,"TA_zips",index=False)
tab_5_tableau_data.to_excel(writer,"tableau_data",index=False)

tab_6_maps_2_plus_stores_TA.to_excel(writer,"maps_2_plus_stores_TA",index=False)
tab_7_maps_1_store_TA_5_miles_plus.to_excel(writer,"maps_1_store_TA_5_miles_plus",index=False)

writer.save()